In [ ]:
# Importando as bibliotecas
import pandas as pd
import sidrar
import openpyxl
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px

# Configurações
fig_width = 6
fig_height = 4

# Conectando ao SIDRA
sidra = sidrar.SIDRA()
ipcacity = sidra.busca(
    tabela=7060,
    localidades="MetroRegion",
    periodos="202302"
)

# IPCA Cesta Básica
cesta_itens = [
    "1101002.Arroz",
    "1101073.Feijão - carioca (rajado)",
    "1114022.Café moído",
    "1104003.Açúcar refinado",
    "1103028.Tomate",
    "1113040.Margarina",
    "1106008.Banana - prata",
    "1107084.Contrafilé",
    "1102023.Farinha de mandioca",
    "1113013.Óleo de soja",
    "1103003.Batata-inglesa",
    "1112015.Pão francês",
    "1111004.Leite longa vida"
]

cesta = ipcacity[ipcacity["Geral, grupo, subgrupo, item e subitem"].isin(cesta_itens)]
cesta = cesta.pivot_table(index=["Região Metropolitana", "Mês"], columns="Variável", values="Valor")
cesta["IPCA Mensal"] = ((cesta["IPCA - Variação mensal"] / 100) + 1) * cesta["IPCA - Peso mensal"]
cesta["IPCA Ano"] = ((cesta["IPCA - Variação acumulada no ano"] / 100) + 1) * cesta["IPCA - Peso mensal"]
cesta["IPCA 12 Meses"] = ((cesta["IPCA - Variação acumulada em 12 meses"] / 100) + 1) * cesta["IPCA - Peso mensal"]
cesta = cesta.groupby(["Região Metropolitana", "Mês"]).sum(numeric_only=True)
cesta["soma peso"] = cesta["IPCA - Peso mensal"].groupby(["Região Metropolitana", "Mês"]).transform("sum")
cesta["IPCA - Variação mensal"] = ((cesta["IPCA Mensal"] / cesta["soma peso"]) - 1) * 100
cesta["IPCA - Variação acumulada no ano"] = ((cesta["IPCA Ano"] / cesta["soma peso"]) - 1) * 100
cesta["IPCA - Variação acumulada em 12 meses"] = ((cesta["IPCA 12 Meses"] / cesta["soma peso"]) - 1) * 100

# Visualização da tabela
print(cesta)

# IPCA Cesta Básica - Gráfico
grafico = cesta.query("`Região Metropolitana` == 'Curitiba - PR' & `Variável` == 'IPCA - Variação acumulada em 12 meses' & Mês == 'fevereiro 2023'")
grafico["Geral, grupo, subgrupo, item e subitem"] = grafico["Geral, grupo, subgrupo, item e subitem"].str.replace(r'\d+', '')
grafico["Geral, grupo, subgrupo, item e subitem"] = grafico["Geral, grupo, subgrupo, item e subitem"].str.replace(".", "")
grafico = grafico.set_index("Geral, grupo, subgrupo, item e subitem")

grafico.plot(kind="barh", figsize=(fig_width, fig_height))
plt.title("IPCA - Cesta Básica Curitiba")
plt.xlabel("Variação Acumulada em 12 meses")
plt.show()

# IPCA Administrados
administrados_itens = [
    "2101004.Taxa de água e esgoto",
    "2201004.Gás de botijão",
    "2202003.Energia elétrica residencial",
    "5102004.Emplacamento e licença",
    "5101001.Ônibus urbano",
    "5104001.Gasolina",
    "6203001.Plano de saúde",
    "6101.Produtos farmacêuticos",
    "5101002.Táxi"
]

administrados = ipcacity[ipcacity["Geral, grupo, subgrupo, item e subitem"].isin(administrados_itens)]
administrados = administrados.pivot_table(index=["Região Metropolitana", "Mês"], columns="Variável", values="Valor")
administrados["IPCA Mensal"] = ((administrados["IPCA - Variação mensal"] / 100) + 1) * administrados["IPCA - Peso mensal"]
administrados["IPCA Ano"] = ((administrados["IPCA - Variação acumulada no ano"] / 100) + 1) * administrados["IPCA - Peso mensal"]
administrados["IPCA 12 Meses"] = ((administrados["IPCA - Variação acumulada em 12 meses"] / 100) + 1) * administrados["IPCA - Peso mensal"]
administrados = administrados.groupby(["Região Metropolitana", "Mês"]).sum(numeric_only=True)
administrados["soma peso"] = administrados["IPCA - Peso mensal"].groupby(["Região Metropolitana", "Mês"]).transform("sum")
administrados["IPCA - Variação mensal"] = ((administrados["IPCA Mensal"] / administrados["soma peso"]) - 1) * 100
administrados["IPCA - Variação acumulada no ano"] = ((administrados["IPCA Ano"] / administrados["soma peso"]) - 1) * 100
administrados["IPCA - Variação acumulada em 12 meses"] = ((administrados["IPCA 12 Meses"] / administrados["soma peso"]) - 1) * 100

# Visualização da tabela
print(administrados)

# IPCA Administrados - Gráfico
grafico_adm = administrados.query("`Região Metropolitana` == 'Curitiba - PR' & `Variável` == 'IPCA - Variação acumulada em 12 meses' & Mês == 'fevereiro 2023'")
grafico_adm["Geral, grupo, subgrupo, item e subitem"] = grafico_adm["Geral, grupo, subgrupo, item e subitem"].str.replace(r'\d+', '')
grafico_adm["Geral, grupo, subgrupo, item e subitem"] = grafico_adm["Geral
